# LAB 4 : Data cleansing

| Member Name                         | Student ID           |
|----------------------------------------|-----------------------|
|Kanyaluck    Chimchome            |    64070501003   |
|Boonyarit     Samran                 |    64070501028   |  	
|Warisara      Patib                     |   64070501044    |  	
|Chanidapa   Chanama                |  64070501090    |	
|Nontawat      Kunlayawuttipong  |  64070501093   	|

## importing libraries and content files

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [90]:
cd_card = pd.read_csv("credit_card_transaction/credit_card_info.csv")
cd_tst = pd.read_csv("credit_card_transaction/credit_transaction.csv")
cd_user = pd.read_csv("credit_card_transaction/credit_users.csv")
mc_code = pd.read_csv("credit_card_transaction/mcc_codes.csv")

## viewing datatype and missing data observation

### credit card

In [91]:
cd_card.head()

,Unnamed: 0,user_id,card_index,card_brand,card_type,card_number,expires,cvv,has_chip,cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,Visa,Debit,1babcd4449ec29a14074260879c13049,12/2022,623,YES,2,$24295,09/2002,2008,No
1,1,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,1,Visa,Debit,4a3fd33483704236f3af0d3e5a602f4f,12/2020,393,YES,2,$21968,04/2014,2014,No
2,2,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,2,Visa,Debit,ea724c282ec208a9c826c2c6bc33b9c7,02/2024,719,YES,2,$46414,07/2003,2004,No
3,3,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,3,Visa,Credit,e24a12553d70b3b18e2d4b3162d82e13,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,4,Mastercard,Debit (Prepaid),65f39625b0df0d5f38860ab79ed2e713,03/2009,75,YES,1,$28,09/2008,2009,No


In [92]:
cd_card.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             732 non-null    int64 
 1   user_id                732 non-null    object
 2   card_index             732 non-null    int64 
 3   card_brand             732 non-null    object
 4   card_type              732 non-null    object
 5   card_number            732 non-null    object
 6   expires                732 non-null    object
 7   cvv                    732 non-null    int64 
 8   has_chip               732 non-null    object
 9   cards_issued           732 non-null    int64 
 10  credit_limit           732 non-null    object
 11  acct_open_date         732 non-null    object
 12  year_pin_last_changed  732 non-null    int64 
 13  card_on_dark_web       732 non-null    object
dtypes: int64(5), object(9)
memory usage: 471.1 KB


### credit transaction

In [93]:
cd_tst.head()

,Unnamed: 0,transaction_datetime,user_id,card,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,is_fraud
0,0,2002-09-01 06:21:15,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,134.09,Swipe Transaction,aec1f6b6-0713-480c-ba05-2bf31892f39c,La Verne,CA,91750.0,5300,NaN,No
1,1,2002-09-01 06:42:17,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,38.48,Swipe Transaction,9f645ffc-fa0a-44ec-8e60-7533bccf63d9,Monterey Park,CA,91754.0,5411,NaN,No
2,2,2002-09-02 06:22:57,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,120.34,Swipe Transaction,9f645ffc-fa0a-44ec-8e60-7533bccf63d9,Monterey Park,CA,91754.0,5411,NaN,No
3,3,2002-09-02 17:45:54,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,128.95,Swipe Transaction,fc611f1f-7adf-42b3-b741-1749bb07eacf,Monterey Park,CA,91754.0,5651,NaN,No
4,4,2002-09-03 06:23:01,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,104.71,Swipe Transaction,370f2dcc-3168-4b18-a6c6-8e612709695a,La Verne,CA,91750.0,5912,NaN,No


In [94]:
cd_tst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2969557 entries, 0 to 2969556
Data columns (total 13 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Unnamed: 0            int64  
 1   transaction_datetime  object 
 2   user_id               object 
 3   card                  int64  
 4   amount                float64
 5   use_chip              object 
 6   merchant_id           object 
 7   merchant_city         object 
 8   merchant_state        object 
 9   zip                   float64
 10  mcc                   int64  
 11  errors                object 
 12  is_fraud              object 
dtypes: float64(2), int64(3), object(8)
memory usage: 294.5+ MB


In [95]:
cd_tst.isna().sum()

Unnamed: 0                    0
transaction_datetime          0
user_id                       0
card                          0
amount                        0
use_chip                      0
merchant_id                   0
merchant_city                 0
merchant_state           377166
zip                      398187
mcc                           0
errors                  2924122
is_fraud                      0
dtype: int64

In [96]:
cd_tst_valid = cd_tst[cd_tst.errors.isna()]
cd_tst_valid = cd_tst_valid.drop(columns=["errors"]).reindex()

### credit users

In [97]:
cd_user.head()

,Unnamed: 0,user_id,person,date_of_birth,retirement_age,gender,address,apartment,city,state,zipcode,per_capita_income_zipcode,yearly_income_person,total_debt,fico_score,num_credit_cards
0,0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,Hazel Robinson,1966/11/24,66,Female,462 Rose Lane,NaN,La Verne,CA,91750,$29278,$59696,$127613,787,5
1,1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,Sasha Sadr,1966/12/25,68,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,$37891,$77254,$191349,701,5
2,2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,Saanvi Lee,1938/11/25,67,Female,766 Third Drive,NaN,West Covina,CA,91792,$22681,$33483,$196,698,5
3,3,acab43d2-f982-47a1-9b4f-d018d069ae16,Everlee Clark,1957/1/16,63,Female,3 Madison Street,NaN,New York,NY,10069,$163145,$249925,$202328,722,4
4,4,80884e14-f347-48c3-8848-768e4c33d693,Kyle Peterson,1976/9/3,70,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,$53797,$109687,$183855,675,1


In [98]:
cd_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 250 non-null    int64  
 1   user_id                    250 non-null    object 
 2   person                     250 non-null    object 
 3   date_of_birth              250 non-null    object 
 4   retirement_age             250 non-null    int64  
 5   gender                     250 non-null    object 
 6   address                    250 non-null    object 
 7   apartment                  69 non-null     float64
 8   city                       250 non-null    object 
 9   state                      250 non-null    object 
 10  zipcode                    250 non-null    int64  
 11  per_capita_income_zipcode  250 non-null    object 
 12  yearly_income_person       250 non-null    object 
 13  total_debt                 250 non-null    object 

In [99]:
cd_user.isna().sum()

Unnamed: 0                     0
user_id                        0
person                         0
date_of_birth                  0
retirement_age                 0
gender                         0
address                        0
apartment                    181
city                           0
state                          0
zipcode                        0
per_capita_income_zipcode      0
yearly_income_person           0
total_debt                     0
fico_score                     0
num_credit_cards               0
dtype: int64

### mcc_codes

In [100]:
mc_code.head()

,mcc,edited_description,combined_description,usda_description,irs_description,irs_reportable
0,742,Veterinary Services,Veterinary Services,Veterinary Services,Veterinary Services,Yes
1,763,Agricultural Co-operatives,Agricultural Co-operatives,Agricultural Co-operatives,Agricultural Cooperative,Yes
2,780,"Horticultural Services, Landscaping Services","Horticultural Services, Landscaping Services",Horticultural Services,Landscaping Services,Yes
3,1520,General Contractors-Residential and Commercial,General Contractors-Residential and Commercial,General Contractors-Residential and Commercial,General Contractors,Yes
4,1711,Air Conditioning Contractors – Sales and Insta...,Air Conditioning Contractors – Sales and Insta...,Air Conditioning Contractors – Sales and Insta...,"Heating, Plumbing, A/C",Yes


In [101]:
mc_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   mcc                   981 non-null    int64 
 1   edited_description    981 non-null    object
 2   combined_description  973 non-null    object
 3   usda_description      706 non-null    object
 4   irs_description       963 non-null    object
 5   irs_reportable        963 non-null    object
dtypes: int64(1), object(5)
memory usage: 46.1+ KB


In [102]:
mc_code.isna().sum()

mcc                       0
edited_description        0
combined_description      8
usda_description        275
irs_description          18
irs_reportable           18
dtype: int64

## Coding 

In [103]:
# result dataframe
user_prof = pd.DataFrame()

In [104]:
user_prof["user_id"] = cd_user.user_id

1. Ages in year

In [105]:
# get referrence date from max transaction date
ref_date = max( pd.to_datetime(cd_tst.transaction_datetime) )
ref_date

Timestamp('2020-02-28 23:49:03')

In [106]:
user_prof["current_age"] = (ref_date - pd.to_datetime(cd_user.date_of_birth)).dt.days // 365
user_prof.head(5)

,user_id,current_age
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63
4,80884e14-f347-48c3-8848-768e4c33d693,43


2. Number of active and expired cards

In [107]:
# Change to datetime type
tempDate = pd.to_datetime(cd_card["expires"], format="%m/%Y")

# Find difference with ref_date in day unit
tempExp = pd.DataFrame((tempDate - ref_date).dt.days)
tempExp["user_id"] = cd_card["user_id"]

# Check did card expire
tempExp["expired_cards"] = tempExp.expires < 0

# Count expired and non-expired
tempExp = tempExp.groupby(['user_id','expired_cards'])['expires'].agg(['count']).reset_index()

# Change    False   -> active_cards
#           True    -> expired_cards
tempExp['expired_cards'] = tempExp['expired_cards'].map({False : 'active_cards', True: 'expired_cards' })

# Change value expired_cards to be columns
tempExp = tempExp.pivot_table(values='count', index='user_id', columns='expired_cards', fill_value=0).reset_index()

tempExp.head(5)

expired_cards,user_id,active_cards,expired_cards
0,01598693-3b8a-424c-b7e1-e7f6a1119702,2,1
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,4,1
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,5,0
3,03625f24-31c9-4e60-b3ad-4d7625985890,5,0
4,03a0d364-2e63-4360-8080-18fb5db7596e,1,1


In [108]:
# merge to user_prof
user_prof = pd.merge(user_prof, tempExp, on="user_id", how='left')
# user_prof.head()

3. Total credit limit of active cards

In [109]:
# Credit_limit_active
tempActiveCard = pd.DataFrame((tempDate - ref_date).dt.days)
tempActiveCard["user_id"] = cd_card['user_id']
tempActiveCard["active_cards"] = tempActiveCard.expires > 0

# Copy Dataframe
tempCl = tempActiveCard.copy()

# Add Dataframe
tempCl['credit_limit_active'] = cd_card['credit_limit']
tempCl['credit_limit_active'] = tempCl.credit_limit_active.str.extract('(\d+)').astype(int)

# Drop expired card
tempCl = tempCl[tempCl.active_cards!=False]

# Each user_id have 2 type of cards -> active / expired
# and different amount of them
tempCl = tempCl.groupby(['user_id'])['credit_limit_active'].agg(['sum']).unstack(fill_value=0).reset_index(name='credit_limit_active')

tempCl.loc[:, ['user_id','credit_limit_active']].head(5)

,user_id,credit_limit_active
0,01598693-3b8a-424c-b7e1-e7f6a1119702,17800
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,73383
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,26251
3,03625f24-31c9-4e60-b3ad-4d7625985890,45013
4,03a0d364-2e63-4360-8080-18fb5db7596e,25977


In [110]:
# Merge in to user_prof
user_prof = pd.merge(user_prof, tempCl.loc[:, ['user_id','credit_limit_active']], on = "user_id", how='left')
# user_prof.head()

4. Time since first card openned (acct_open_date) in months

In [111]:
# Create Dataframe to keep user_id and acct_open_date from credit_card_info
tempOC = pd.DataFrame()
tempOC["user_id"] = cd_card['user_id']
tempOC["time_since_first_card_months"] = pd.to_datetime(cd_card['acct_open_date'])

# Select min_date_time of acct_open_date in each user_id
tempOC = tempOC.groupby(["user_id"])['time_since_first_card_months'].min().reset_index()

# ref_date - min_date_time in each user_id
# and turn value in to Month
tempOC["time_since_first_card_months"] = (ref_date - tempOC['time_since_first_card_months']) // np.timedelta64(1, 'M')

tempOC.head(5)

,user_id,time_since_first_card_months
0,01598693-3b8a-424c-b7e1-e7f6a1119702,157
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,240
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,222
3,03625f24-31c9-4e60-b3ad-4d7625985890,133
4,03a0d364-2e63-4360-8080-18fb5db7596e,154


In [112]:
# Merge in to user_prof
user_prof = pd.merge(user_prof,tempOC, on = "user_id", how='left')
# user_prof.head()

5. Time since last transaction in days

In [113]:
# Create Dataframe
tempLast = pd.DataFrame()
tempLast["user_id"] = cd_tst['user_id']
tempLast["time_since_last_transaction_days"] = pd.to_datetime(cd_tst['transaction_datetime'])

# Select max_date_time of last transaction day in each user_id
tempLast = tempLast.groupby(["user_id"])['time_since_last_transaction_days'].max().reset_index()

# ref_date - max_date_time in each user_id
# and turn value in to Day
tempLast['time_since_last_transaction_days'] = (ref_date - tempLast['time_since_last_transaction_days']) // np.timedelta64(1, 'D')

tempLast.head(5)

,user_id,time_since_last_transaction_days
0,01598693-3b8a-424c-b7e1-e7f6a1119702,0
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,0
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,0
3,03625f24-31c9-4e60-b3ad-4d7625985890,1
4,03a0d364-2e63-4360-8080-18fb5db7596e,1


In [114]:
# Merge in to user_prof
user_prof = pd.merge(user_prof,tempLast, on = "user_id", how='left')
# user_prof.head()

6. Distribution of active card type (Debit, Credit, Prepaid)

In [115]:
tempDate = pd.to_datetime(cd_card["expires"], format="%m/%Y")

# Find difference with ref_date in day unit
tempExp = pd.DataFrame((tempDate - ref_date).dt.days)
tempExp["user_id"] = cd_card["user_id"]
tempExp["card_type"] = cd_card["card_type"]

# Check did card expire
tempExp["expired_cards"] = tempExp.expires < 0
tempExp = tempExp[~tempExp["expired_cards"]]
testtypecard = tempExp.pivot_table(index = "user_id", columns = "card_type", values = "expired_cards", aggfunc = "count", fill_value=0)


# Rename columns
# Credit             ->     active_card_credit
# Debit              ->     active_card_debit
# Debit (Prepaid)    ->     active_card_debit_(prepaid)
testtypecard = testtypecard.rename(columns={ 'Credit': 'active_card_credit', 'Debit': 'active_card_debit', 'Debit (Prepaid)': 'active_card_debit_(prepaid)' })

testtypecard.head(5)

card_type,active_card_credit,active_card_debit,active_card_debit_(prepaid)
user_id,,,
01598693-3b8a-424c-b7e1-e7f6a1119702,2,0,0
01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,1,3,0
0326d7bb-0c79-4523-905f-c068bb4a3ebf,1,4,0
03625f24-31c9-4e60-b3ad-4d7625985890,2,3,0
03a0d364-2e63-4360-8080-18fb5db7596e,0,1,0


In [116]:
# Merge in to user_prof
user_prof = pd.merge(user_prof,testtypecard, on = "user_id", how='left')
# user_prof.head()

7. Number of merchants that this user made transaction

In [117]:
# Create Dataframe to keep user_id and merchant_id from credit_transaction
tempM = pd.DataFrame()
tempM["user_id"] = cd_tst['user_id']
tempM["num_merchants"] = cd_tst['merchant_id']

# Each user_id have different number of merchants
tempM = tempM.groupby(["user_id"]).agg({"num_merchants":"nunique"}).reset_index()

tempM.head(5)

,user_id,num_merchants
0,01598693-3b8a-424c-b7e1-e7f6a1119702,258
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,532
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,349
3,03625f24-31c9-4e60-b3ad-4d7625985890,166
4,03a0d364-2e63-4360-8080-18fb5db7596e,310


In [118]:
# Merge in to user_prof
user_prof = pd.merge(user_prof,tempM, on = "user_id", how='left')
# user_prof.head()

8. Number of states that this user made transaction

In [119]:
# Create Dataframe to keep user_id and merchant_state from credit_transaction
tempstates = pd.DataFrame()
tempstates["user_id"] = cd_tst['user_id']
tempstates["num_states"] = cd_tst['merchant_state']

# Each user_id have different number of states
tempstates = tempstates.groupby(["user_id"]).agg({"num_states":"nunique"}).reset_index()

tempstates.head(5)

,user_id,num_states
0,01598693-3b8a-424c-b7e1-e7f6a1119702,30
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,38
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,35
3,03625f24-31c9-4e60-b3ad-4d7625985890,21
4,03a0d364-2e63-4360-8080-18fb5db7596e,28


In [120]:
# Merge in to user_prof
user_prof = pd.merge(user_prof,tempstates, on = "user_id", how='left')
# user_prof.head()

9. Number of times that user made transaction outside the USA (online transaction counts as nonUSA transaction)

In [121]:
# Create Dataframe to keep user_id and merchant_city from credit_transaction
tempnonUSA = pd.DataFrame()
tempnonUSA["user_id"] = cd_tst['user_id']
tempnonUSA["abroad_tx"] = cd_tst['merchant_city']

# Check if value in abroad_tx is "ONLINE" then return True
#       if not return False
tempnonUSA['abroad_tx'] = tempnonUSA['abroad_tx'].isin(['ONLINE'])

# Then keep row that return True
tempnonUSA = tempnonUSA[tempnonUSA.abroad_tx == True]

# Each user_id have different nonUSA transaction times
tempnonUSA = tempnonUSA.groupby(["user_id"]).agg({"abroad_tx":"size"}).reset_index()

tempnonUSA.head(5)

,user_id,abroad_tx
0,01598693-3b8a-424c-b7e1-e7f6a1119702,13234
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,742
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,939
3,03625f24-31c9-4e60-b3ad-4d7625985890,417
4,03a0d364-2e63-4360-8080-18fb5db7596e,610


In [122]:
# Merge in to user_prof
user_prof = pd.merge(user_prof,tempnonUSA, on = "user_id",how = "left")
# user_prof.head()

10. All-time transaction count

In [123]:
# all_time_tx_count
temp = cd_tst.groupby(['user_id'])\
    .agg({'user_id':'count'})\
    .rename(columns={'user_id': 'all_time_tx_count'})\
    .reset_index()

temp.head(5)

,user_id,all_time_tx_count
0,01598693-3b8a-424c-b7e1-e7f6a1119702,23152
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,24921
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,26125
3,03625f24-31c9-4e60-b3ad-4d7625985890,4174
4,03a0d364-2e63-4360-8080-18fb5db7596e,13251


In [124]:
# add all_time_tx_count to user_prof
user_prof = user_prof.merge(temp, how = 'left',on = 'user_id')
# user_prof.head()

11. All-time amount used

In [125]:
# all_time_amount_used
temp = cd_tst_valid.groupby(['user_id'])\
    .agg({'amount':sum})\
    .rename(columns={'amount':'all_time_amount_used'})\
    .reset_index()

temp.head(5)

,user_id,all_time_amount_used
0,01598693-3b8a-424c-b7e1-e7f6a1119702,840783.54
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,1281003.48
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,586612.68
3,03625f24-31c9-4e60-b3ad-4d7625985890,209340.88
4,03a0d364-2e63-4360-8080-18fb5db7596e,527795.97


In [126]:
# add all_time_amount_used to user_prof
user_prof = user_prof.merge(temp, how = 'left',on = 'user_id')
# user_prof.head()

12. All-time median amount per transaction

In [127]:
# All-time median amount per transaction
temp = cd_tst_valid.groupby(['user_id'])\
    .agg({'amount': 'median'})\
    .rename(columns={'amount':'all_time_median_amount_per_tx'})\
    .reset_index()

temp.head(5)

,user_id,all_time_median_amount_per_tx
0,01598693-3b8a-424c-b7e1-e7f6a1119702,35.020
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,43.300
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,15.540
3,03625f24-31c9-4e60-b3ad-4d7625985890,31.705
4,03a0d364-2e63-4360-8080-18fb5db7596e,40.860


In [128]:
# add all_time_median_amount_per_tx to user_prof
user_prof = user_prof.merge(temp, how = 'left',on = 'user_id')
# user_prof.head()

13. All-time number of fraud transaction

In [129]:
# All-time number of fraud transaction
temp = cd_tst[cd_tst['is_fraud'] == 'Yes'][['user_id', 'is_fraud']]

temp = temp.groupby('user_id')\
    .agg({'is_fraud': 'count'})\
    .rename(columns={'is_fraud':'all_time_fraud_tx_count'})\
    .reset_index()

temp.head(5)

,user_id,all_time_fraud_tx_count
0,01598693-3b8a-424c-b7e1-e7f6a1119702,5
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,43
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,26
3,03625f24-31c9-4e60-b3ad-4d7625985890,1
4,03a0d364-2e63-4360-8080-18fb5db7596e,29


In [130]:
# add all_time_fraud_tx_count to user_prof
user_prof = user_prof.merge(temp, how = 'left',on = 'user_id')
user_prof['all_time_fraud_tx_count'] = user_prof['all_time_fraud_tx_count'].fillna(0) # NaN mean they never fraud( fraud always 'No')
# user_prof.head()

14. All-time number of refunded transaction (negative amount)

In [131]:
# get only refunded transction
temp = cd_tst[cd_tst['amount'] < 0 ][['user_id', 'amount']]

# All-time number of refunded transaction (negative amount)
temp = temp.groupby(['user_id'])\
    .agg({'amount': 'count'})\
    .rename(columns={'amount':'all_time_refunded_tx_count'})\
    .reset_index()

temp.head(5)


,user_id,all_time_refunded_tx_count
0,01598693-3b8a-424c-b7e1-e7f6a1119702,727
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,647
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,1153
3,03625f24-31c9-4e60-b3ad-4d7625985890,301
4,03a0d364-2e63-4360-8080-18fb5db7596e,747


In [132]:
# add all_time_refunded_tx_count to user_prof
user_prof = user_prof.merge(temp, how = 'left',on = 'user_id')
del temp
# user_prof.head()

15. All-time fraud transaction percentage

In [133]:
# copy columns cd_tst['user_id'] and cd_tst['is_fraud'] to aftp (temp) dataframe
# use not clean transaction df because consider all transaction
aftp = pd.DataFrame()
aftp['user_id'] = cd_tst['user_id']
aftp['is_fraud'] = cd_tst['is_fraud']

# count each user which fraud 'Yes' and 'No'
aftp = aftp[[ 'user_id', 'is_fraud' ]].value_counts().reset_index(name='count')

# count all transaction
aftp_u = aftp.groupby(['user_id'])['count'].agg(['sum']).reset_index()

# count only is_fraud == 'Yes'
aftp_u = pd.merge(aftp_u, aftp[ aftp['is_fraud'] == 'Yes' ], on='user_id')\
            .rename(columns={'count' : 'fraud'})\
            .drop(columns=['is_fraud'])

# find percentage from ( <is_fraud ==yes> / <all transaction> ) * 100
aftp_u['percent'] = (aftp_u['fraud'] / aftp_u['sum']) * 100

aftp_u.head(5)


,user_id,sum,fraud,percent
0,01598693-3b8a-424c-b7e1-e7f6a1119702,23152,5,0.021596
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,24921,43,0.172545
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,26125,26,0.099522
3,03625f24-31c9-4e60-b3ad-4d7625985890,4174,1,0.023958
4,03a0d364-2e63-4360-8080-18fb5db7596e,13251,29,0.218851


In [134]:
# merge to user_prof
user_prof = pd.merge(user_prof, aftp_u.loc[:, ['user_id','percent']].rename(columns={'percent' : 'all_time_fraud_tx_percentage'})
, on="user_id", how='left')
# user_prof.head(10)

16. All-time median duration between transaction in hours

In [135]:
# copy columns cd_tst_valid['user_id'] and cd_tst_valid['transaction_datetime'] to amdbt (temp) dataframe
# use clean transaction df because consider median duration of transaction that success
amdbt = pd.DataFrame()
amdbt['user_id'] = cd_tst_valid['user_id']
amdbt['transaction_datetime'] = pd.to_datetime(cd_tst_valid['transaction_datetime'])

# sort value (for grouping it more easily and for shift data to previous transaction)
amdbt.sort_values(by=['user_id', 'transaction_datetime'], inplace=True)

# shift id and datetime to compare with the previous transaction
amdbt['id_shift'] = amdbt['user_id'].shift(1)
amdbt['time_shift'] = amdbt['transaction_datetime'].shift(1)

# delete row that id does not same with previous one (mean first transaction of each id because no time to compare)
amdbt.drop(amdbt[ amdbt['user_id'] != amdbt['id_shift'] ].index, inplace=True)
amdbt.drop(columns=['id_shift'], inplace=True)      # no need shift id for next method

# let `diff` columns be the delta of duration between transaction then change type of time to hour
amdbt['diff'] = (amdbt['transaction_datetime'] - amdbt['time_shift']).astype('timedelta64[h]')

# find the median of duration between transaction from each user
amdbt = amdbt.groupby(['user_id'])['diff'].agg(all_time_median_time_between_tx_hours='median').reset_index()

amdbt.head(5)

,user_id,all_time_median_time_between_tx_hours
0,01598693-3b8a-424c-b7e1-e7f6a1119702,1.0
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,3.0
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,3.0
3,03625f24-31c9-4e60-b3ad-4d7625985890,17.0
4,03a0d364-2e63-4360-8080-18fb5db7596e,5.0


In [136]:
# merge to user_prof
user_prof = pd.merge(user_prof, amdbt, on="user_id", how='left')
# user_prof.head(10)

17. All-time average monthly transaction count (do not consider month with no transaction)

In [137]:
# copy columns cd_tst['user_id'] and cd_tst['transaction_datetime'] to aamtc (temp) dataframe
# use not clean transaction df because consider all transaction
aamtc = pd.DataFrame()
aamtc['user_id'] = cd_tst['user_id']
aamtc['transaction_datetime'] = pd.to_datetime(cd_tst['transaction_datetime'])

# create column value with default 1 (for count)
aamtc['value'] = 1

# group by user_id and month then count number of transaction
aamtc = aamtc.groupby(['user_id', pd.Grouper(freq='1M', key='transaction_datetime')])['value'].agg(['count']).reset_index()

# drop if that month have no transaction
aamtc.drop(aamtc[aamtc['count'] == 0].index)

# find mean of monthly count transaction of each user
aamtc = aamtc.groupby(['user_id'])['count'].agg(all_time_avg_monthly_tx_count='mean').reset_index()

aamtc.head(5)

,user_id,all_time_avg_monthly_tx_count
0,01598693-3b8a-424c-b7e1-e7f6a1119702,145.610063
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,103.406639
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,117.152466
3,03625f24-31c9-4e60-b3ad-4d7625985890,31.149254
4,03a0d364-2e63-4360-8080-18fb5db7596e,85.490323


In [138]:
# merge to user_prof
user_prof = pd.merge(user_prof, aamtc, on="user_id", how='left')
# user_prof.head(10)

18. All-time average monthly amount (do not consider month with no transaction)

In [139]:
# copy columns cd_tst_valid['user_id'], cd_tst_valid['transaction_datetime'] and cd_tst_valid['amount'] to aama (temp) dataframe
# use clean transaction df because consider only success transaction (amount not include error amount)
aama = pd.DataFrame()
aama['user_id'] = cd_tst_valid['user_id']
aama['transaction_datetime'] = pd.to_datetime(cd_tst_valid['transaction_datetime'])
aama['amount'] = cd_tst_valid['amount']

# group by user_id and month then sum the amount of transaction
aama = aama.groupby(['user_id', pd.Grouper(freq='1M', key='transaction_datetime')]).agg('sum').reset_index()

# find mean monthly amount of each user
aama = aama.groupby(['user_id'])['amount'].agg(all_time_avg_monthly_amount='mean').reset_index()

aama.head(5)

,user_id,all_time_avg_monthly_amount
0,01598693-3b8a-424c-b7e1-e7f6a1119702,5287.946792
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,5315.367137
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,2630.550135
3,03625f24-31c9-4e60-b3ad-4d7625985890,1562.245373
4,03a0d364-2e63-4360-8080-18fb5db7596e,3405.135290


In [140]:
# merge to user_prof
user_prof = pd.merge(user_prof, aama, on="user_id", how='left')
# user_prof.head(10)

19. All-time transaction method distribution ( use_chip column)

In [141]:
# copy columns cd_tst['user_id'] and cd_tst['use_chip'] to atmd (temp) dataframe
# use not clean transaction df because consider all transaction method
atmd = pd.DataFrame()
atmd['user_id'] = cd_tst['user_id']
atmd['use_chip'] = cd_tst['use_chip']

# count number of each method (of each user)
atmd = atmd.groupby(['user_id' , 'use_chip'])['use_chip'].agg(['count']).reset_index()

# rename in column `use_chip`
#   Chip Transaction    -> all_time_tx_method_chip_transaction
#   Online Transaction  -> all_time_tx_method_online_transaction
#   Swipe Transaction   -> all_time_tx_method_swipe_transaction
atmd['use_chip'] = atmd['use_chip'].map({
    'Chip Transaction': 'all_time_tx_method_chip_transaction', 
    'Online Transaction': 'all_time_tx_method_online_transaction', 
    'Swipe Transaction': 'all_time_tx_method_swipe_transaction'})

# change count in `use_chip` to be columns by pivot table
atmd = atmd.pivot_table(values='count', index='user_id', columns='use_chip', fill_value=0).reset_index()

atmd.head(5)

use_chip,user_id,all_time_tx_method_chip_transaction,all_time_tx_method_online_transaction,all_time_tx_method_swipe_transaction
0,01598693-3b8a-424c-b7e1-e7f6a1119702,3564,13233,6355
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,5009,742,19170
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,5619,938,19568
3,03625f24-31c9-4e60-b3ad-4d7625985890,1144,415,2615
4,03a0d364-2e63-4360-8080-18fb5db7596e,4219,606,8426


In [142]:
# merge to user_prof
user_prof = pd.merge(user_prof, atmd.loc[:, ::1], on="user_id", how='left')
# user_prof.head(10)

#### Create Last Calendar Year Dataframe

In [144]:
# let's find last year data (In 2019)

# copy transaction data to tst_lcy
tst_lcy = cd_tst.copy()

# extract year from transaction_datetime and pick rows if their year are 2019
tst_lcy['transaction_datetime'] = pd.to_datetime(tst_lcy['transaction_datetime'])
tst_lcy['Year'] = tst_lcy['transaction_datetime'].dt.year
tst_lcy = tst_lcy.loc[tst_lcy['Year'] == ref_date.year - 1]

# result
tst_lcy.head(5)

,Unnamed: 0,transaction_datetime,user_id,card,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,is_fraud,Year
4776,4776,2019-01-05 06:02:56,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,120.30,Chip Transaction,e12b0ab7-167a-4dc3-81ed-4a3fa74db901,Princeton,MA,1541.0,5411,Insufficient Balance,No,2019
4777,4777,2019-01-06 20:10:10,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,68.43,Chip Transaction,ea16e3d6-9543-410d-ac25-c741401ebec2,North Grafton,MA,1536.0,5812,NaN,No,2019
4778,4778,2019-01-09 10:18:03,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,59.17,Chip Transaction,fa234243-ab93-4f26-beaa-15de51b3cb90,North Grafton,MA,1536.0,7538,NaN,No,2019
4779,4779,2019-01-11 13:13:45,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,16.07,Chip Transaction,e12b0ab7-167a-4dc3-81ed-4a3fa74db901,Princeton,MA,1541.0,5411,NaN,No,2019
4780,4780,2019-01-12 05:54:35,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,132.82,Chip Transaction,60e4760a-eb92-45d9-a6ad-979c221c36eb,Sturbridge,MA,1566.0,5300,NaN,No,2019


In [145]:
# drop year column and drop rows that have errors 
tst_lcy.drop(columns='Year', inplace=True)
tst_lcy_dperr = tst_lcy[tst_lcy.errors.isna()]

# result
tst_lcy_dperr.head(5)

,Unnamed: 0,transaction_datetime,user_id,card,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,is_fraud
4777,4777,2019-01-06 20:10:10,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,68.43,Chip Transaction,ea16e3d6-9543-410d-ac25-c741401ebec2,North Grafton,MA,1536.0,5812,NaN,No
4778,4778,2019-01-09 10:18:03,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,59.17,Chip Transaction,fa234243-ab93-4f26-beaa-15de51b3cb90,North Grafton,MA,1536.0,7538,NaN,No
4779,4779,2019-01-11 13:13:45,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,16.07,Chip Transaction,e12b0ab7-167a-4dc3-81ed-4a3fa74db901,Princeton,MA,1541.0,5411,NaN,No
4780,4780,2019-01-12 05:54:35,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,132.82,Chip Transaction,60e4760a-eb92-45d9-a6ad-979c221c36eb,Sturbridge,MA,1566.0,5300,NaN,No
4782,4782,2019-01-13 00:20:23,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,-265.00,Chip Transaction,e0174d7f-77e8-4867-80f0-efe15dadce35,Oxford,MA,1540.0,3504,NaN,No


20. Last calendar year transaction count

In [146]:
temp_trans = tst_lcy[['user_id']].copy()
# count transaction by user_id
temp_trans = temp_trans.groupby(['user_id']).agg({'user_id':'count'})
temp_trans = temp_trans.rename(columns={'user_id':'last_year_tx_count'}).reset_index()

temp_trans.head(5)

,user_id,last_year_tx_count
0,01598693-3b8a-424c-b7e1-e7f6a1119702,1971
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,1295
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,1423
3,03625f24-31c9-4e60-b3ad-4d7625985890,386
4,03a0d364-2e63-4360-8080-18fb5db7596e,1004


In [147]:
# merge to user_prof
user_prof = pd.merge(user_prof, temp_trans, on="user_id", how='left').fillna(0)
user_prof = user_prof.astype({'last_year_tx_count' : 'int64'})
# user_prof.head(10)

21. Last calendar year amount used

In [148]:
# use dataframe that have already drop the error transaction to find Last calendar amount used
temp_amount = tst_lcy_dperr[['user_id','amount']].copy()
temp_amount = temp_amount.groupby(['user_id']).agg({'amount':'sum'}).reset_index()
temp_amount = temp_amount.rename(columns={'amount':'last_year_amount_used'})

temp_amount.head(5)

,user_id,last_year_amount_used
0,01598693-3b8a-424c-b7e1-e7f6a1119702,72833.22
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,57730.52
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,29028.52
3,03625f24-31c9-4e60-b3ad-4d7625985890,20040.78
4,03a0d364-2e63-4360-8080-18fb5db7596e,38533.61


In [149]:
# merge 2 dataframe together by user_id
user_prof = pd.merge(user_prof, temp_amount, on="user_id", how='left').fillna(0)
# user_prof.head(10)

22. Last calendar year median amount per transaction

In [150]:
# use dataframe that have already drop the error transaction to find Last calendar amount used
temp_amount_md = tst_lcy_dperr[['user_id','amount']].copy()
temp_amount_md = temp_amount_md.groupby(['user_id']).agg({'amount':'median'}).reset_index()
temp_amount_md = temp_amount_md.rename(columns={'amount':'last_year_median_amount_per_tx'})

temp_amount_md.head(5)

,user_id,last_year_median_amount_per_tx
0,01598693-3b8a-424c-b7e1-e7f6a1119702,35.540
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,18.610
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,9.535
3,03625f24-31c9-4e60-b3ad-4d7625985890,32.820
4,03a0d364-2e63-4360-8080-18fb5db7596e,40.430


In [151]:
# merge 2 dataframe together by user_id
user_prof = pd.merge(user_prof, temp_amount_md, on="user_id", how='left').fillna(0)
# user_prof.head(10)

23. Last calendar year number of fraud transaction

In [152]:
temp_fraud = tst_lcy[['user_id','is_fraud']].copy()

# filter fraud transaction
temp_fraud = temp_fraud[temp_fraud.is_fraud == 'Yes']

# count fraud transaction of each user
temp_fraud = temp_fraud.groupby(['user_id']).agg({'is_fraud':'count'}).reset_index()
temp_fraud = temp_fraud.rename(columns={'is_fraud':'last_year_fraud_tx_count'})

temp_fraud.head(5)

,user_id,last_year_fraud_tx_count
0,17c6936c-9a0e-4b65-b6de-d800112ca82a,7
1,34342ce3-e578-4047-bb36-3fdd15a97ae9,13
2,40d899de-aa9e-428c-84a1-503fa36e18c4,15
3,40da85ec-19fc-464e-aa6a-7f7e7544e847,8
4,44f79937-8da3-4131-b155-fc1dad54f324,14


In [153]:

# merge 2 dataframe together by user_id
user_prof = pd.merge(user_prof, temp_fraud, on="user_id", how='left').fillna(0)
# user_prof.head(10)

24. Last calendar year number of refunded transaction (negative amount)

In [154]:
temp_refunded = tst_lcy[['user_id','amount']].copy()

# use only the negative amount
temp_refunded = temp_refunded[temp_refunded.amount < 0]
temp_refunded = temp_refunded.groupby(['user_id']).agg({'amount':'count'}).reset_index()
temp_refunded = temp_refunded.rename(columns={'amount':'last_year_refunded_tx_count'})

temp_refunded.head(5)

,user_id,last_year_refunded_tx_count
0,01598693-3b8a-424c-b7e1-e7f6a1119702,51
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,30
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,51
3,03625f24-31c9-4e60-b3ad-4d7625985890,18
4,03a0d364-2e63-4360-8080-18fb5db7596e,54


In [155]:
# merge 2 dataframe together by user_id
user_prof = pd.merge(user_prof, temp_refunded, on="user_id", how='left').fillna(0)
# user_prof.head(10)

25. Last calendar year fraud transaction percentage

In [156]:
user_prof['last_year_fraud_tx_percentage'] = (user_prof['last_year_fraud_tx_count'] / user_prof['last_year_tx_count']) * 100
user_prof = user_prof.fillna(0)
# user_prof.head(10)

26. Last calendar year median duration between transaction in hours

In [157]:
temp_duration = tst_lcy[['user_id','transaction_datetime']].copy()

# sort data before find diff
temp_duration = temp_duration.sort_values(by = ['user_id','transaction_datetime'])

# find duration between transaction in hours
temp_duration['transaction_diff'] = temp_duration.groupby(['user_id'])['transaction_datetime'].diff()/ pd.Timedelta(hours=1)
temp_duration = temp_duration.dropna()
temp_duration['transaction_diff'] = temp_duration['transaction_diff'].apply(np.floor)

# find median duration between transaction in hours
temp_duration = temp_duration.groupby(['user_id']).agg({'transaction_diff':'median'}).reset_index()
temp_duration = temp_duration.rename(columns={'transaction_diff':'last_year_median_time_between_tx_hours'})
# temp_duration['last_year_median_time_between_tx_hours'] = temp_duration['last_year_median_time_between_tx_hours'].apply(np.floor)

temp_duration.head(5)

,user_id,last_year_median_time_between_tx_hours
0,01598693-3b8a-424c-b7e1-e7f6a1119702,1.0
1,01c0d742-95e2-4e9a-a9e9-f7f0ec82ecb0,2.0
2,0326d7bb-0c79-4523-905f-c068bb4a3ebf,2.0
3,03625f24-31c9-4e60-b3ad-4d7625985890,16.0
4,03a0d364-2e63-4360-8080-18fb5db7596e,5.0


In [158]:
user_prof = pd.merge(user_prof, temp_duration, on="user_id", how='left').fillna(0)
# user_prof.head(10)

30. Last calendar year average monthy transaction count (do not consider month with no transaction)
31. Last calendar year average monthly amount (do not consider month with no transaction) <br />
**we will simultaneously do these two execises using `pivot_table()`**

In [159]:
# create 5 samples of user_id for output correction   
exList = [  "4e98b4b9-f30f-4117-bf0b-49c6b550aba6",
            "8e283e3b-fb65-4d2a-9c7e-b515c94d8402",
            "8c9b9223-c41c-4999-9cff-f72b9c4bcf7c",
            "acab43d2-f982-47a1-9b4f-d018d069ae16",
            "80884e14-f347-48c3-8848-768e4c33d693"
]

In [160]:
# use pivot_table to find sumation and counting "amount" of any last year users. 
acc_tab = tst_lcy_dperr.pivot_table(index = "user_id", values = "amount", aggfunc={np.sum, "count"}).reset_index()
# acc_tab

In [161]:
# extract month from transaction_datetime and look into pivot table to count which months have order
tst_lcy_dperr["Month"] = pd.DatetimeIndex(tst_lcy_dperr["transaction_datetime"]).strftime("%m")
monthDistri = tst_lcy_dperr.pivot_table(index = ["user_id"],columns = "Month", values = "amount", aggfunc = "count").reset_index()
acc_tab["en_month"] = 12-monthDistri.isna().sum(axis = 1)
#acc_tab

C:\Users\User\AppData\Local\Temp\ipykernel_9188\1941781117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tst_lcy_dperr["Month"] = pd.DatetimeIndex(tst_lcy_dperr["transaction_datetime"]).strftime("%m")


In [162]:
# commpute averages by order-actived month dividing
acc_tab["last_year_avg_monthly_tx_count"] = acc_tab["count"] / acc_tab["en_month"]
acc_tab["last_year_avg_monthly_amount"] = acc_tab["sum"] / acc_tab["en_month"]

# check other who aren't order every months
#acc_tab[acc_tab["en_month"] != 12]

# result
#acc_tab

In [163]:
# check with samples
acc_tab[acc_tab.user_id.isin(exList)]

,user_id,count,sum,en_month,last_year_avg_monthly_tx_count,last_year_avg_monthly_amount
66,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,1128,90807.49,12,94.000000,7567.290833
93,80884e14-f347-48c3-8848-768e4c33d693,912,88372.25,12,76.000000,7364.354167
101,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,2529,85076.19,12,210.750000,7089.682500
102,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,553,45710.35,12,46.083333,3809.195833
131,acab43d2-f982-47a1-9b4f-d018d069ae16,847,119715.90,12,70.583333,9976.325000


In [164]:
# drop dummies columns
acc_tab.drop(columns = ["count", "sum", "en_month"], inplace = True)
#acc_tab

In [165]:
# merge to user_prof
user_prof = pd.merge(user_prof, acc_tab, on = "user_id", how = "left")

# testing with samples
user_prof[user_prof["user_id"].isin(exList)]

,user_id,current_age,active_cards,expired_cards,credit_limit_active,time_since_first_card_months,time_since_last_transaction_days,active_card_credit,active_card_debit,active_card_debit_(prepaid),...,all_time_tx_method_swipe_transaction,last_year_tx_count,last_year_amount_used,last_year_median_amount_per_tx,last_year_fraud_tx_count,last_year_refunded_tx_count,last_year_fraud_tx_percentage,last_year_median_time_between_tx_hours,last_year_avg_monthly_tx_count,last_year_avg_monthly_amount
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,105077.0,209,0,1.0,3.0,0.0,...,15840,1164,90807.49,64.840,0.0,35.0,0.000000,5.0,94.000000,7567.290833
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,37683.0,199,1,0.0,2.0,2.0,...,5691,566,45710.35,69.240,10.0,31.0,1.766784,9.0,46.083333,3809.195833
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,106597.0,215,0,0.0,4.0,0.0,...,20316,2573,85076.19,36.210,0.0,31.0,0.000000,2.0,210.750000,7089.682500
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,258224.0,156,0,0.0,2.0,1.0,...,5169,860,119715.90,58.580,0.0,41.0,0.000000,5.0,70.583333,9976.325000
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,34900.0,242,0,0.0,1.0,0.0,...,12495,921,88372.25,88.735,0.0,11.0,0.000000,5.0,76.000000,7364.354167


32. Last calendar year transaction method distribution (`use_chip column`)

In [166]:
# count "amount" on specific "use_chip" types of any "user_id"
tst_distr = tst_lcy_dperr.pivot_table(index = "user_id", columns = "use_chip", values = "amount", aggfunc = "count")\
                         .reset_index()

# merge to user_prof
user_prof = pd.merge(user_prof, tst_distr, on = "user_id", how = "left")

# rename columns
user_prof.rename(columns = {"Chip Transaction": "last_year_tx_method_chip_transaction",
                            "Online Transaction": "last_year_tx_method_online_transaction" ,
                            "Swipe Transaction" : "last_year_tx_method_swipe_transaction"}, inplace =True)

In [167]:
# testing with samples
user_prof[user_prof["user_id"].isin(exList)]

,user_id,current_age,active_cards,expired_cards,credit_limit_active,time_since_first_card_months,time_since_last_transaction_days,active_card_credit,active_card_debit,active_card_debit_(prepaid),...,last_year_median_amount_per_tx,last_year_fraud_tx_count,last_year_refunded_tx_count,last_year_fraud_tx_percentage,last_year_median_time_between_tx_hours,last_year_avg_monthly_tx_count,last_year_avg_monthly_amount,last_year_tx_method_chip_transaction,last_year_tx_method_online_transaction,last_year_tx_method_swipe_transaction
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,105077.0,209,0,1.0,3.0,0.0,...,64.840,0.0,35.0,0.000000,5.0,94.000000,7567.290833,551.0,74.0,503.0
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,37683.0,199,1,0.0,2.0,2.0,...,69.240,10.0,31.0,1.766784,9.0,46.083333,3809.195833,495.0,27.0,31.0
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,106597.0,215,0,0.0,4.0,0.0,...,36.210,0.0,31.0,0.000000,2.0,210.750000,7089.682500,1000.0,1122.0,407.0
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,258224.0,156,0,0.0,2.0,1.0,...,58.580,0.0,41.0,0.000000,5.0,70.583333,9976.325000,721.0,93.0,33.0
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,34900.0,242,0,0.0,1.0,0.0,...,88.735,0.0,11.0,0.000000,5.0,76.000000,7364.354167,710.0,143.0,59.0


33. Top merchant category (MCC) based on number of transactions

In [168]:
# these result aren't the same as Sample output (because dropping error transactions)
cd_tst_temp = cd_tst[cd_tst.errors.isna()]

# find Top merchant category, counting on specified mcc of any user id
tempDf = cd_tst_temp.pivot_table(index = "user_id",columns = "mcc", values = "amount", aggfunc = "count").reset_index()
#tempDf["top_mcc_by_tx_count_times"] = tempDf.drop(columns=tempDf.columns[0]).max(axis = 1)
tempDf["mcc"] = tempDf.drop(columns=[tempDf.columns[0],tempDf.columns[-1]]).idxmax(axis =1)

# merging and renaming columns
tempDf = pd.merge(tempDf[["user_id","mcc"]], mc_code[["mcc", "edited_description"]], on = "mcc", how = "left")
tempDf = tempDf.rename(columns = {"mcc":"top_mcc_by_tx_count","edited_description":"top_mcc_by_tx_count_desc"})
user_prof = pd.merge(user_prof, tempDf, on="user_id", how = "left")

In [169]:
# testing with samples
user_prof[user_prof["user_id"].isin(exList)]

,user_id,current_age,active_cards,expired_cards,credit_limit_active,time_since_first_card_months,time_since_last_transaction_days,active_card_credit,active_card_debit,active_card_debit_(prepaid),...,last_year_refunded_tx_count,last_year_fraud_tx_percentage,last_year_median_time_between_tx_hours,last_year_avg_monthly_tx_count,last_year_avg_monthly_amount,last_year_tx_method_chip_transaction,last_year_tx_method_online_transaction,last_year_tx_method_swipe_transaction,top_mcc_by_tx_count,top_mcc_by_tx_count_desc
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,105077.0,209,0,1.0,3.0,0.0,...,35.0,0.000000,5.0,94.000000,7567.290833,551.0,74.0,503.0,5411,"Grocery Stores, Supermarkets"
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,37683.0,199,1,0.0,2.0,2.0,...,31.0,1.766784,9.0,46.083333,3809.195833,495.0,27.0,31.0,7538,Automotive Service Shops
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,106597.0,215,0,0.0,4.0,0.0,...,31.0,0.000000,2.0,210.750000,7089.682500,1000.0,1122.0,407.0,4784,Toll and Bridge Fees
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,258224.0,156,0,0.0,2.0,1.0,...,41.0,0.000000,5.0,70.583333,9976.325000,721.0,93.0,33.0,5411,"Grocery Stores, Supermarkets"
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,34900.0,242,0,0.0,1.0,0.0,...,11.0,0.000000,5.0,76.000000,7364.354167,710.0,143.0,59.0,5311,Department Stores


34. Top merchant category (MCC) based on amount paid

In [170]:
# these result aren't the same as Sample output (because dropping error transactions)
cd_tst_temp = cd_tst[cd_tst.errors.isna()]

# find Top merchant category, sum on specified mcc of any user id
tempDf = cd_tst_temp.pivot_table(index = "user_id",columns = "mcc", values = "amount", aggfunc = "sum").reset_index()
tempDf["mcc"] = tempDf.drop(columns=[tempDf.columns[0],tempDf.columns[-1]]).idxmax(axis =1)

# merging and renaming columns
tempDf = pd.merge(tempDf[["user_id","mcc"]], mc_code[["mcc", "edited_description"]], on = "mcc", how = "left")
tempDf = tempDf.rename(columns = {"mcc":"top_mcc_by_tx_amount","edited_description":"top_mcc_by_tx_amount_desc"})
user_prof = pd.merge(user_prof, tempDf, on="user_id", how = "left")

In [171]:
# testing with samples
user_prof[user_prof["user_id"].isin(exList)]

,user_id,current_age,active_cards,expired_cards,credit_limit_active,time_since_first_card_months,time_since_last_transaction_days,active_card_credit,active_card_debit,active_card_debit_(prepaid),...,last_year_median_time_between_tx_hours,last_year_avg_monthly_tx_count,last_year_avg_monthly_amount,last_year_tx_method_chip_transaction,last_year_tx_method_online_transaction,last_year_tx_method_swipe_transaction,top_mcc_by_tx_count,top_mcc_by_tx_count_desc,top_mcc_by_tx_amount,top_mcc_by_tx_amount_desc
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,105077.0,209,0,1.0,3.0,0.0,...,5.0,94.000000,7567.290833,551.0,74.0,503.0,5411,"Grocery Stores, Supermarkets",5411,"Grocery Stores, Supermarkets"
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,37683.0,199,1,0.0,2.0,2.0,...,9.0,46.083333,3809.195833,495.0,27.0,31.0,7538,Automotive Service Shops,7538,Automotive Service Shops
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,106597.0,215,0,0.0,4.0,0.0,...,2.0,210.750000,7089.682500,1000.0,1122.0,407.0,4784,Toll and Bridge Fees,4784,Toll and Bridge Fees
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,258224.0,156,0,0.0,2.0,1.0,...,5.0,70.583333,9976.325000,721.0,93.0,33.0,5411,"Grocery Stores, Supermarkets",4829,Money Orders – Wire Transfer
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,34900.0,242,0,0.0,1.0,0.0,...,5.0,76.000000,7364.354167,710.0,143.0,59.0,5311,Department Stores,5311,Department Stores


35. Number of each error types (one transaction can have multiple errors)

In [172]:
# get transactions which come with errors
cd_tst_err = cd_tst[cd_tst.errors.notna()]

# handling multiple errors in a cell
tempDf = cd_tst_err[["user_id", "errors", "amount"]]
tempDf["errors"] = tempDf["errors"].str.split(',')
tempDf = tempDf.explode("errors", ignore_index=True)

# np.unique(tempDf["errors"]) # recheck error has been seperated

# counting each errors type occured to user_id 
err_report = tempDf.pivot_table(index ="user_id", columns = "errors", values= "amount", aggfunc = "count").reset_index()
err_report = err_report.fillna(0)
err_report.columns = ["user_id",
                      "tx_errors_bad_cvv",
                      "tx_errors_bad_card_number",
                      "tx_errors_bad_expiration",
                      "tx_errors_bad_pin",
                      "tx_errors_bad_zipcode",
                      "tx_errors_insufficient_balance",
                      "tx_errors_technical_glitch"]

C:\Users\User\AppData\Local\Temp\ipykernel_9188\3175537786.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDf["errors"] = tempDf["errors"].str.split(',')


In [173]:
# merge to user_prof
user_prof = pd.merge(user_prof, err_report, on = "user_id", how = "left")

# replace Nan to 0 for entire user_prof
user_prof.fillna(0, inplace = True)

In [174]:
# testing with samples
user_prof[user_prof["user_id"].isin(exList)]

,user_id,current_age,active_cards,expired_cards,credit_limit_active,time_since_first_card_months,time_since_last_transaction_days,active_card_credit,active_card_debit,active_card_debit_(prepaid),...,top_mcc_by_tx_count_desc,top_mcc_by_tx_amount,top_mcc_by_tx_amount_desc,tx_errors_bad_cvv,tx_errors_bad_card_number,tx_errors_bad_expiration,tx_errors_bad_pin,tx_errors_bad_zipcode,tx_errors_insufficient_balance,tx_errors_technical_glitch
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,105077.0,209,0,1.0,3.0,0.0,...,"Grocery Stores, Supermarkets",5411,"Grocery Stores, Supermarkets",2.0,4.0,9.0,44.0,0.0,470.0,47.0
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,37683.0,199,1,0.0,2.0,2.0,...,Automotive Service Shops,7538,Automotive Service Shops,3.0,1.0,5.0,23.0,0.0,96.0,18.0
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,106597.0,215,0,0.0,4.0,0.0,...,Toll and Bridge Fees,4784,Toll and Bridge Fees,69.0,80.0,70.0,82.0,1.0,279.0,89.0
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,258224.0,156,0,0.0,2.0,1.0,...,"Grocery Stores, Supermarkets",4829,Money Orders – Wire Transfer,8.0,6.0,2.0,29.0,0.0,37.0,18.0
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,34900.0,242,0,0.0,1.0,0.0,...,Department Stores,5311,Department Stores,6.0,9.0,11.0,63.0,0.0,94.0,32.0


## last result

In [175]:
user_prof.head(10)

,user_id,current_age,active_cards,expired_cards,credit_limit_active,time_since_first_card_months,time_since_last_transaction_days,active_card_credit,active_card_debit,active_card_debit_(prepaid),...,top_mcc_by_tx_count_desc,top_mcc_by_tx_amount,top_mcc_by_tx_amount_desc,tx_errors_bad_cvv,tx_errors_bad_card_number,tx_errors_bad_expiration,tx_errors_bad_pin,tx_errors_bad_zipcode,tx_errors_insufficient_balance,tx_errors_technical_glitch
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,105077.0,209,0,1.0,3.0,0.0,...,"Grocery Stores, Supermarkets",5411,"Grocery Stores, Supermarkets",2.0,4.0,9.0,44.0,0.0,470.0,47.0
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,37683.0,199,1,0.0,2.0,2.0,...,Automotive Service Shops,7538,Automotive Service Shops,3.0,1.0,5.0,23.0,0.0,96.0,18.0
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,106597.0,215,0,0.0,4.0,0.0,...,Toll and Bridge Fees,4784,Toll and Bridge Fees,69.0,80.0,70.0,82.0,1.0,279.0,89.0
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,258224.0,156,0,0.0,2.0,1.0,...,"Grocery Stores, Supermarkets",4829,Money Orders – Wire Transfer,8.0,6.0,2.0,29.0,0.0,37.0,18.0
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,34900.0,242,0,0.0,1.0,0.0,...,Department Stores,5311,Department Stores,6.0,9.0,11.0,63.0,0.0,94.0,32.0
5,709c2790-8f93-4e27-83fb-dad9ea3647c3,42,2,1,25700.0,217,0,2.0,0.0,0.0,...,Money Orders – Wire Transfer,4829,Money Orders – Wire Transfer,3.0,5.0,1.0,38.0,0.0,368.0,46.0
6,fddc40fe-3311-4cb4-aaf4-44f0c4085ac7,36,3,0,67917.0,176,0,1.0,2.0,0.0,...,Taxicabs and Limousines,4121,Taxicabs and Limousines,32.0,38.0,36.0,52.0,0.0,128.0,48.0
7,f0f1033b-aaa1-4e81-aa02-ed10b788a08d,26,1,0,17480.0,1,0,0.0,1.0,0.0,...,Eating places and Restaurants,5812,Eating places and Restaurants,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8,ca371403-8abe-459b-873b-0aa1e61557a5,81,4,1,74599.0,181,0,2.0,2.0,0.0,...,Toll and Bridge Fees,5300,Wholesale Clubs,29.0,29.0,33.0,28.0,0.0,246.0,44.0
9,eda2429c-2528-4ab0-9ba0-ec0f541433db,34,1,0,9496.0,127,0,0.0,1.0,0.0,...,Service Stations ( with or without ancillary s...,5541,Service Stations ( with or without ancillary s...,0.0,0.0,4.0,74.0,0.0,180.0,38.0


In [176]:
user_prof.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 48 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   user_id                                 250 non-null    object 
 1   current_age                             250 non-null    int64  
 2   active_cards                            250 non-null    int64  
 3   expired_cards                           250 non-null    int64  
 4   credit_limit_active                     250 non-null    float64
 5   time_since_first_card_months            250 non-null    int64  
 6   time_since_last_transaction_days        250 non-null    int64  
 7   active_card_credit                      250 non-null    float64
 8   active_card_debit                       250 non-null    float64
 9   active_card_debit_(prepaid)             250 non-null    float64
 10  num_merchants                           250 non-null    int64 

## Source Code
[Github link -> https://github.com/Querians/Data_Cleansing](https://github.com/Querians/Data_Cleansing)